### Correction of the ML-Ready Cell-Painting Data Sets
#### Prerequisite
This notebook is part of the master thesis of Luis Vollmers. It uses the cell paintin data set of Bray et al. and the goal of the project is to predict pubchem assay data. Originally this notebook was concerned with pubchem AID 1030 but this might be subject to change.
#### Introduction
The datasets generated in ml_ready of step 01 comprise several mistakes that have been made and that need to be corrected within this jupyter notebook. Mostly, the compounds listed in the rows of the ML-ready dataframes are not unique, due to a misunderstanding stemming from the Bray et al Cell-Painting paper. It was stated that the Metadata_broad_sample is a unique identifier which was found to be wrong. Identical compounds which were used in different concentrations and/or on different well-plates were assigned different Metadata_broad_sample values. This needs to be corrected as well as the averaging over the different column values. In the data columns, the cell painting features are listed and so far the algorithm just kept the first concentration and took the average of the respective multiplicates. The correct way of doing it however is to check how many concentrations are present and then take the median of the concentration that is most frequent which is done by this jupyter notebook. 
#### Summary of the steps in this notebook
1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
2. Reintroduce the center median data into the pubchem df
3. Treat the multi-concentration Compounds adequately
4. Export the output


In [1]:
import pandas as pd

#### 1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
- Inputs are taken from the directory for step 1 of the pubchem assays and from the preprocessing directory
- the data from cp_1030 is erroneous but the meta data is needed for the overlap with the center median data
- therefore filter the df for the metadata

In [9]:
pd.set_option("display.max_rows", 300)
# define the meta cols of the pubchem assays that are relevant and filter the df 
meta_cols = ['Metadata_broad_sample','PUBCHEM_ACTIVITY_OUTCOME', 'PUBCHEM_ACTIVITY_SCORE']

# load the data into RAM 'cp_1030' can be used as a variable for a bash script
dataset_path='../../01-FilteringAssays/ml_ready/cp_504847.csv'
df = pd.read_csv(dataset_path, usecols=meta_cols)

# read the cell-painting data into RAM
cp = pd.read_csv('../../../preprocessing/cp_center_median.csv', index_col=0)

#### 2. Reintroduce the Center Median Data into the Pubchem DF
- the common column is Metadata_broad_sample which was responsible for the errors in the first place
- it is actually the only identifier column that is left in the ML-ready df
- the inner merge makes sure that only rows are kept that can be found in both dataframes

In [10]:
# merge the cp and the assay data on the common column
merged = pd.merge(left=cp, right=df, on='Metadata_broad_sample', how='inner')

In [11]:
# this cell calculates the number of rows expected from the final dataframe
multi_list = []
concs_list = merged.CAN_SMILES.value_counts().to_list()

for i in concs_list:
    if i > 1:
        multi_list.append(i)
        
merged.shape[0]-sum(multi_list)+len(multi_list)

8454

#### 3. Treat the multi-concentration Compounds adequately
- this step is the only one that needs to be conducted manually
- first look at the multi conc compounds and determine how to treat them
- in this case the concentrations that are less frequent are dropped and the remaining ones are calculated into median
- for that purpose the merged DF is split into data and meta columns and only the data columns are treated accordingly
- the groupby method is used to calculate the compound wise median 
- the single concentrations are already calculated so only the multi concs are actually computed herein

In [18]:
# have a look at the multi concentration compounds and figure out how to treat them
merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']] # only uncomment if you havent checked for said compounds yet

,CAN_SMILES,Metadata_mmoles_per_liter
9179,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9180,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9181,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9182,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9183,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9184,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9185,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9186,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9187,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9188,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507


In [19]:
merged = merged.drop(merged[(merged.SINGLE_CONC==False) & (merged.Metadata_mmoles_per_liter > 4.0)].index)

In [20]:
# Quality Control step that makes sure only the relevant rows with the most concentrations are kept
merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']] # just needed as a one time lookup

,CAN_SMILES,Metadata_mmoles_per_liter
9179,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9180,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9181,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9182,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9183,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9184,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9185,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9186,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9187,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
9188,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507


In [21]:
# define a list of the all columns of the merged data frame
all_cols = merged.columns.to_list()
# redefine meta columns with the meta information of the cell painting assay
meta_cols = ['CAN_SMILES','CPD_SMILES','Metadata_broad_sample','Metadata_Plate_Map_Name','Metadata_ASSAY_WELL_ROLE','Metadata_Plate','SINGLE_CONC','PUBCHEM_ACTIVITY_SCORE','PUBCHEM_ACTIVITY_OUTCOME']

In [22]:
# data cols are basically all columns without the meta data. hence the forloop that removes those from data_cols
data_cols = all_cols
for item in meta_cols:
    data_cols.remove(item)
    
# afterwards the 'CAN_SMILES' column is inserted at the first position
data_cols.insert(0,'CAN_SMILES')

In [23]:
# the merged data frame is split into two dataframes containing meta and raw data information
merged_data = merged.loc[:,data_cols]
merged_meta = merged.loc[:,meta_cols]

In [24]:
# this command takes the compound wise median of the data
merged_data = merged_data.groupby('CAN_SMILES').median().reset_index()

#### 4. Export the Output
- as a last step the only thing that needs to be done is to merge the meta and data columns back into one DF
- a bit of a clean up needs to be done since the merge command creates suplicates, which can be safely deleted
- output in csv format named according to the pubchem AID

In [25]:
# the median data is merged back with the meta data
merged = pd.merge(left=merged_meta, right=merged_data, on='CAN_SMILES', how='left')

In [26]:
# merging generally keeps all rows in both frames so that duplicates are generated, which get hereby deleted
merged = merged.drop_duplicates(subset='CAN_SMILES')

In [27]:
merged.to_csv('../_output/cp_504847.csv',index=False)

In [28]:
pd.read_csv('../_output/cp_504847.csv') # only uncomment for quality control purposes, i.e. visual conformation

,CAN_SMILES,CPD_SMILES,Metadata_broad_sample,Metadata_Plate_Map_Name,Metadata_ASSAY_WELL_ROLE,Metadata_Plate,SINGLE_CONC,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_OUTCOME,Metadata_mmoles_per_liter,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,COc1ccc2c(=O)c(CC(=O)O)coc2c1,COc1ccc2c(c1)occ(CC(O)=O)c2=O,BRD-K78414110-001-02-8,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,0.028752,-0.141545,-0.091271,-0.123436,-0.234958,-0.261659,-0.269276,0.162881,0.061376,0.104751
1,CCCc1nc(C(C)(C)O)c(C(=O)OCc2oc(=O)oc2C)n1Cc1cc...,CCCc1nc(C(C)(C)O)c(C(=O)OCc2oc(=O)oc2C)n1Cc1cc...,BRD-K78485176-001-02-9,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,-0.046285,-0.177514,-0.196004,-0.227765,-0.182558,-0.212246,-0.204336,0.087412,0.048356,0.082538
2,COc1cc(OC)c2ccc(=O)oc2c1,COc1cc(OC)c2ccc(=O)oc2c1,BRD-K78612426-001-02-6,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,-0.061549,-0.133824,-0.130796,-0.131496,-0.153068,-0.156526,-0.125545,0.107378,0.023594,0.045900
3,COC(=O)c1ccccc1NC(=O)c1ccc(F)cc1,COC(=O)c1ccccc1NC(=O)c1ccc(F)cc1,BRD-K78633253-001-01-2,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,0.098785,-0.029717,-0.036163,-0.049478,-0.122305,-0.091394,-0.115040,0.200919,0.124473,0.153696
4,O=C(CCCN1CCC(O)(c2ccc(Br)cc2)CC1)c1ccc(F)cc1,OC1(CCN(CCCC(=O)c2ccc(F)cc2)CC1)c1ccc(Br)cc1,BRD-K78643075-001-03-3,H-BIOA-007-3,treated,24278,True,0.0,Inactive,2.379167,...,0.208695,-0.024429,-0.044756,-0.048945,-0.091941,-0.096263,-0.109370,0.053506,0.038483,0.040109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8449,CCOC(=O)c1c(C)oc2c1c(C(c1cccnc1)N1CCCCC1)c(O)c...,CCOC(=O)c1c(C)oc2c1c(C(N1CCCCC1)c1cccnc1)c(O)c...,BRD-A43050491-001-05-6,C-2113-01-D39-007,treated,26204,True,0.0,Inactive,5.000000,...,0.003111,0.057371,0.013895,0.005887,-0.003589,0.081811,0.062094,-0.116301,-0.099438,-0.107922
8450,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,Cn1c(cc(=O)c2ccccc12)-c1ccc2OCOc2c1,BRD-K92683369-001-03-6,H-BIOA-007-3,treated,24278,False,0.0,Inactive,3.580507,...,-0.104311,0.005627,-0.037484,-0.027687,0.023850,-0.032037,0.004835,-0.078895,-0.036355,-0.064550
8451,CC(C)(C)c1ccc(C(O)CCCN2CCC(C(O)(c3ccccc3)c3ccc...,CC(C)(C)c1ccc(cc1)C(O)CCCN1CCC(CC1)C(O)(c1cccc...,BRD-A06352418-001-03-6,H-BIOA-001-3,treated,24294,False,0.0,Inactive,2.120111,...,-0.031586,0.058706,0.032472,0.043662,0.006015,0.019518,0.049757,-0.014844,-0.015225,-0.025579
8452,COC(=O)Nc1nc2cc(SC(C)C)ccc2[nH]1,COC(=O)Nc1nc2cc(SC(C)C)ccc2[nH]1,BRD-K02764365-001-03-7,H-BIOA-003-3,treated,24305,False,0.0,Inactive,3.768872,...,0.027589,0.329196,0.292009,0.379255,-0.098949,-0.160635,-0.177683,0.025531,-0.251917,-0.232001
